In [13]:
import os
import os.path
import datetime
import pandas as pd
import time
from datetime import datetime
import pysftp
from func_timeout import func_timeout

Directory where the files are saved and create the list of all files in directory

In [14]:
'''directory'''
search_dir = f'C:\Users\{user}\Documents'

'''list all files'''
files = os.listdir(search_dir)

create a list with name and date of alteration (file)

In [16]:
list_files = {None : None}
for file in files:
    date = datetime.fromtimestamp(os.path.getmtime(search_dir + file)).strftime('%d/%m/%Y %H:%M:%S')
    new_item = {file : date}
    list_files.update(new_item)

we created the list of files what will is update in WinSCP

In [17]:
up_files = []

for file in list_files.keys():
    if file is not None:
        day_file = int(list_files[file][0:2])
        
        if day_file == pd.Timestamp('now').day:
            hour_file = int(list_files[file][11:13])
            hour_now = int(pd.Timestamp('now').hour)

            if hour_now >= 8 and hour_now < 15 and hour_file >= 8 and hour_file < 15:
                up_files.append(file)
            elif hour_now >= 15 and hour_now < 24 and hour_file >= 15 and hour_file < 24:
                up_files.append(file)

we created the list of timers for each size file

In [18]:
list_size = {None : None}
new_item = None

for file in files:

    size = os.path.getsize(search_dir + file) * 0.0009765625

    if size <= 10000:
        new_item = {file : 30}
        list_size.update(new_item)
    elif size > 10000 and size <= 50000:
        new_item = {file : 60}
        list_size.update(new_item)
    elif size > 50000 and size <= 100000:
        new_item = {file : 90}
        list_size.update(new_item)
    elif size > 100000 and size <= 150000:
        new_item = {file : 180}
        list_size.update(new_item)
    elif size > 150000 and size <= 300000:
        new_item = {file : 300}
        list_size.update(new_item)
    elif size > 300000:
        new_item = {file : 360}
        list_size.update(new_item)

Now, we connect with WinSCP and we upload the files in the server

In [19]:
def connection_server(file, file_up, server, user, key, port, cnopts):
    with pysftp.Connection(host = server, username = user, password = key, port = port,  cnopts = cnopts) as sftp:
        print('Connection successfully')

        with sftp.cd('user/share/prod/'):

            print(f'{file} loading...')

            sftp.put(file_up, confirm = False, preserve_mtime=False)

            print(f'connection and load of file {file} successful.')

            time.sleep(3)
    sftp.close()

In [ ]:
server = 'host'
user = 'my_user'
key = 'my_password'
port = 22

cnopts = pysftp.CnOpts()
cnopts.hostkeys = None
sec_timeout = 0

for file in up_files:

    file_up = search_dir + file
    sec_timeout = list_size[file]
    print(f'{sec_timeout} seconds')
    
    try:
        func_timeout(sec_timeout, connection_server, args = (file, file_up, server, user, key, port, cnopts))
    except:
        print(f'{file} time out of execution after {sec_timeout}')
### upar no prod